In [64]:
import pandas as pd
vix_daily = pd.read_csv("Data/VIXCLS-2.csv", parse_dates=["observation_date"], index_col="observation_date")
vix_daily.rename(columns={"observation_date": "date"}, inplace=True)
vix_daily.rename(columns={"VIXCLS": "VIX"}, inplace=True)
vix = vix_daily.resample('ME').mean()
vix.index = vix.index.to_period('M').to_timestamp('M')
vix.index.name ="date"
returns_dataframe = pd.read_csv('Data/long_short_returns_clean.csv', index_col='date')
returns = pd.DataFrame(returns_dataframe['long_short_return'])
returns.index = pd.to_datetime(returns.index)
returns.index = returns.index.to_period('M').to_timestamp('M')

credit_spread_daily = pd.read_csv("Data/BAA10Y.csv", parse_dates=['observation_date'], index_col='observation_date')
credit_spread = credit_spread_daily.resample('ME').mean()
credit_spread.index = credit_spread.index.to_period('M')
credit_spread = credit_spread/100
credit_spread.index.name = 'date'
credit_spread.index = credit_spread.index.to_timestamp()
credit_spread.index = credit_spread.index.to_period('M').to_timestamp('M')
credit_spread.rename(columns={"BAA10Y": "Credit_Spread"}, inplace=True)


df = pd.concat([returns, vix, credit_spread], axis=1, join="inner")
df

,long_short_return,VIX,Credit_Spread
date,,,
1990-01-31,0.013870,23.347273,0.017386
1990-02-28,-0.008700,23.262632,0.016711
1990-03-31,-0.013850,20.062273,0.016182
1990-04-30,-0.008110,21.403500,0.015160
1990-05-31,-0.034014,18.097727,0.016532
...,...,...,...
2018-02-28,-0.001459,22.464737,0.016500
2018-03-31,0.018048,19.023810,0.017976
2018-04-30,0.011210,18.267619,0.018000


In [67]:
import pandas as pd
import statsmodels.api as sm


# Define a high risk aversion dummy based on VIX (e.g., above median)
df['high_vix'] = (df['VIX'] > df['VIX'].median()).astype(int)

# Interaction term
df['value_x_vix'] = df['long_short_return'] * df['high_vix']

# High credit spread dummy (above median)
df['high_cs'] = (df['Credit_Spread'] > df['Credit_Spread'].median()).astype(int)

# Interaction with value premium
df['value_x_cs'] = df['long_short_return'] * df['high_cs']

# Regression: value premium on high_vix dummy and interaction
#X = sm.add_constant(df[['high_vix', 'value_x_vix']])
X = sm.add_constant(df[['high_vix', 'value_x_vix', 'high_cs', 'value_x_cs']])
y = df['long_short_return']

model = sm.OLS(y, X).fit(cov_type="HC3")
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:      long_short_return   R-squared:                       0.845
Model:                            OLS   Adj. R-squared:                  0.843
Method:                 Least Squares   F-statistic:                     779.0
Date:                Thu, 26 Jun 2025   Prob (F-statistic):          8.00e-169
Time:                        14:54:07   Log-Likelihood:                 1055.6
No. Observations:                 342   AIC:                            -2101.
Df Residuals:                     337   BIC:                            -2082.
Df Model:                           4                                         
Covariance Type:                  HC3                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0039      0.001      3.068      

In [68]:
high_vix_returns = df.loc[df['high_vix'] == 1, 'long_short_return']
low_vix_returns = df.loc[df['high_vix'] == 0, 'long_short_return']

print("Mean Value Premium when VIX is High:", high_vix_returns.mean())
print("Mean Value Premium when VIX is Low:", low_vix_returns.mean())


Mean Value Premium when VIX is High: 0.003403270179035231
Mean Value Premium when VIX is Low: 0.003491890063284098


The average difference in raw returns between just “high VIX” vs. “low VIX” months (ignoring factor levels) is indeed very small (~0.1%).
However, the effect of the value premium on returns is much stronger in the high VIX regime.
In other words, risk aversion regimes don’t change the baseline returns much directly, but they amplify or dampen how much the value factor translates into actual returns.